<a href="https://colab.research.google.com/github/olorin44/dw_matrix_road_sign/blob/master/part3_day5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install hyperopt

In [15]:
import pandas as pd
import numpy as np
import tensorflow as tf
import os
import datetime

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

%load_ext tensorboard

import matplotlib.pyplot as plt
from skimage import color, exposure
from sklearn.metrics import accuracy_score

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [5]:
cd '/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_three/dw_matrix_road_sign'

/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_three/dw_matrix_road_sign


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

x_train, y_train = train['features'], train['labels']
x_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

input_shape = x_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(get_model, x_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(
      x_train, 
      y_train,
      batch_size=params_fit.get('batch_size', 128),
      epochs=params_fit.get('epochs', 5),
      verbose=params_fit.get('verbose', 1),
      validation_data=params_fit.get('validation_data', (x_train, y_train)),
      callbacks= [tensorboard_callback]
  )

  return model

def predict(model_trained, x_test, y_test, scoring=accuracy_score):

  y_test_norm = np.argmax(y_test, axis=1)

  y_pred_prob = model_trained.predict(x_test)
  y_pred = np.argmax(y_pred_prob, axis=1)

  return scoring(y_test_norm, y_pred)

In [0]:
def get_cnn_v5(input_shape, num_classes):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(0.3),

    Flatten(),

    Dense(1024, activation='relu'),
    Dropout(0.3),

    Dense(1024, activation='relu'),
    Dropout(0.3),

    Dense(num_classes, activation='softmax'),
  ])


In [12]:
model = get_cnn_v5(input_shape, num_classes)
model_trained = train_model(model, x_train, y_train)
predict(model_trained, x_test, y_test)

Epoch 1/5
272/272 [==============================] - 5s 18ms/step - loss: 2.5995 - accuracy: 0.2959 - val_loss: 1.1815 - val_accuracy: 0.6553
Epoch 2/5
272/272 [==============================] - 5s 17ms/step - loss: 0.8987 - accuracy: 0.7146 - val_loss: 0.2407 - val_accuracy: 0.9370
Epoch 3/5
272/272 [==============================] - 4s 16ms/step - loss: 0.3920 - accuracy: 0.8807 - val_loss: 0.1087 - val_accuracy: 0.9684
Epoch 4/5
272/272 [==============================] - 4s 16ms/step - loss: 0.2630 - accuracy: 0.9195 - val_loss: 0.0589 - val_accuracy: 0.9843
Epoch 5/5
272/272 [==============================] - 4s 16ms/step - loss: 0.1983 - accuracy: 0.9416 - val_loss: 0.0576 - val_accuracy: 0.9828


0.9498866213151927

In [0]:
def get_model(params):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_one']),

    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_two']),

    Conv2D(filters=128, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
    Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_three']),

    Flatten(),

    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_one']),

    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_two']),

    Dense(num_classes, activation='softmax'),
  ])


In [0]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  model.fit(
      x_train, 
      y_train,
      batch_size=int(params.get('batch_size', 128)),
      epochs=5,
      verbose=0,
  )

  score = model.evaluate(x_test, y_test, verbose=0)
  accuracy = score[1]
  print(params, 'accuracy={}'.format(accuracy))

  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}

In [44]:
space = {
    'batch_size': hp.quniform('batch_size', 100, 200, 10),
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0.3, 0.5),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0.3, 0.5),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0.3, 0.5),

    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one', 0.3, 0.7),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two', 0.3, 0.7),
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

{'batch_size': 110.0, 'dropout_cnn_block_one': 0.4240827796211221, 'dropout_cnn_block_three': 0.4397020574487186, 'dropout_cnn_block_two': 0.37671291152318775, 'dropout_dense_block_one': 0.4331692863685499, 'dropout_dense_block_two': 0.34350836495747955}
accuracy=0.960544228553772
{'batch_size': 200.0, 'dropout_cnn_block_one': 0.3394312450760235, 'dropout_cnn_block_three': 0.3713475288629307, 'dropout_cnn_block_two': 0.40342531938609594, 'dropout_dense_block_one': 0.4953562635398995, 'dropout_dense_block_two': 0.5696496513627449}
accuracy=0.9671201705932617
{'batch_size': 160.0, 'dropout_cnn_block_one': 0.3593182427438963, 'dropout_cnn_block_three': 0.305971447927452, 'dropout_cnn_block_two': 0.35654921307645976, 'dropout_dense_block_one': 0.5802557003510715, 'dropout_dense_block_two': 0.39030681622717656}
accuracy=0.9501133561134338
{'batch_size': 180.0, 'dropout_cnn_block_one': 0.3750927219097241, 'dropout_cnn_block_three': 0.46876845689007635, 'dropout_cnn_block_two': 0.382950304647